In [ ]:
%reload_ext autoreload
%autoreload 2
import networkqit as nq
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import scipy
# import some dependencies
import torch
import pyro
import pyro.distributions as dist

In [ ]:
A = nx.to_numpy_array(nx.karate_club_graph())
L = nq.graph_laplacian(A)
beta_range=np.logspace(1,-4,5)
N=len(A)
print(nx.density(nx.karate_club_graph()))
M = nq.ErdosRenyi(N=N)
def sampling_er(p):
    return nx.to_numpy_array(nx.erdos_renyi_graph(N,p))

In [ ]:
x0=np.random.random(1,)
#x0=np.array([0.2,])
solver = nq.Adam(A=A,x0=x0,beta_range=np.logspace(3,-3,25))
print('Starting from',solver.x0)
solver.setup(expected_adj_fun = M, adj_sampling_fun = sampling_er, expected_laplacian_grad=M.expected_laplacian_grad, step_callback = lambda beta,p : print('\rbeta=',beta,'p=',p[0],end=''))
sol,all_x = solver.run(max_iters=200,alpha=1E-3,num_samples=50)

In [ ]:
x0=np.random.random(1,)
#x0=np.array([0.2,])
solver = nq.StochasticGradientDescent(A=A,x0=x0,beta_range=np.logspace(1,-1,25))
print('Starting from',solver.x0)
solver.setup(expected_adj_fun = M, adj_sampling_fun = sampling_er, expected_laplacian_grad=M.expected_laplacian_grad, step_callback = lambda beta,p : print('\rbeta=',beta,'p=',p[0],end=''))
sol,all_x = solver.run(clip_gradients=(-1000,1000),max_iters=200,eta=1E-3,num_samples=50)

In [ ]:
plt.plot(np.array([nx.density(nx.karate_club_graph())]*len(all_x)))
plt.plot(all_x)

In [ ]:
import pandas as pd
df = pd.DataFrame(sol)

#df.plot(x='T',y='rel_entropy',logx=True,logy=True,figsize=(12,12))
plt.semilogx(np.array(df['T']),np.array([nx.density(nx.karate_club_graph())]*len(df)))
#plt.semilogx(np.array(df['T']),[x[0] for x in df['x']])
#plt.xlabel('1/beta')
#plt.legend(['$E[S(\\rho,\\sigma)]$','$p^*$','$\tilde{p}$'])

In [ ]:
solver = nq.StochasticGradientDescent(A=A,x0=np.random.random(1,),beta_range=beta_range,clip=(-1,1))
solver.setup(expected_adj_fun = M, adj_sampling_fun = sampling_er, expected_laplacian_grad=M.expected_laplacian_grad, step_callback = lambda beta,p: print('\rbeta=',beta,'p=',p,end=''))
sol = solver.run(clip_gradients=(-1,1),num_iters=1000,eta=1E-3,num_samples=1000)

In [ ]:
from scipy.linalg import eigvalsh
from scipy.special import logsumexp

In [ ]:
def ER(n,p):
    return nq.graph_laplacian(nx.to_numpy_array(nx.erdos_renyi_graph(n,p)))

In [ ]:
p=0.5
beta=1
def average_eigs(n,p,n_samples):
    l = np.array(np.zeros([n]))
    for i in range(0,n_samples):
        l += eigvalsh(ER(n,p))
    return l/n_samples

In [ ]:
N=500
x=[]
samples=range(1,5,1)
for nsamples in samples:
    x.append(nd.Derivative(lambda x : average_eigs(N,x,nsamples))(0.5))
    print(nsamples)

In [ ]:
sns.set_palette(sns.cubehelix_palette(len(samples)))
plt.figure(figsize=(12,8))
plt.plot(np.array(x))
plt.legend(list(samples))
plt.show()
for y in x:
    print('<x>=',np.mean(y),'+-',np.std(y))

In [ ]:
x=[(logsumexp(-beta*eigvalsh(nq.graph_laplacian(sampling_er(p+eps)))) - logsumexp(-beta*eigvalsh(nq.graph_laplacian(sampling_er(p)))))/eps for i in range(0,1000)]
np.mean(x)
np.std(x)